In [1]:
import torchvision 
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms   
import torch.optim as optim
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from numpy.linalg import norm
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision 
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.nn.utils.prune as prune
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable

In [2]:
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision 

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
transform1 = transforms.Compose(
    [    
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261)),   
    ])
trainset = torchvision.datasets.CIFAR10(root='./', train=True,download=True,transform = transform1)
indices = torch.randperm(len(trainset))[:5000]
trainset1 = torch.utils.data.Subset(trainset, indices)
modelloader = torch.utils.data.DataLoader(trainset, shuffle=True, num_workers=2, batch_size = 100)
validset = torchvision.datasets.CIFAR10(root='./', train=True,download=True, transform = transform) 
valid = list(range(40000, 50000,1))
validset1 = torch.utils.data.Subset(validset, valid)
validloader = torch.utils.data.DataLoader(validset1, shuffle=True, num_workers=4,batch_size = 100)
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 1)
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable


def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                             F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20(classes):
    return ResNet(BasicBlock, [3, 3, 3], classes)




In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 =nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(128 * 2 * 2, 24)
        self.fc2 = nn.Linear(24,2)
       # self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
      #  print(x.size())
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
       # x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 =nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(128 * 2 * 2, 24)
        self.fc2 = nn.Linear(24,3)
       # self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
      #  print(x.size())
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
       # x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x


In [6]:
def layer1transform(array,size):
    newarr = array.detach().clone()
    for i in range(size):
        if array[i] <= 2:
            newarr[i] = 0
        elif array[i] < 6:
            newarr[i] = 1
        else: 
            newarr[i] = 2
    return newarr
def layer2transform1(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        if array[i] <= 2:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] == 0:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 1:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 2: 
                newarr = torch.cat((newarr,ttensor),dim=0)
    return newdata, newarr
def layer2transform2(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        if array[i] >2 and array[i] <= 5:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] == 3:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 4:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 5: 
                newarr = torch.cat((newarr,ttensor),dim=0)
    return newdata, newarr
def layer2transform3(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        thtensor = torch.Tensor([3])
        thtensor = ttensor.to(device)
        if array[i] > 5:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] == 6:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 7:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 8: 
                newarr = torch.cat((newarr,ttensor),dim=0)
            elif array[i] == 9:
                newarr = torch.cat((newarr,thtensor),dim=0)
    return newdata, newarr

def test():
    for data, target in modelloader:
        data, target = data.to(device), target.to(device)
        layer2transform2(data,target)
    

    
    

In [5]:
def layer1transform(array,size):
    newarr = array.detach().clone()
    for i in range(size):
        if array[i] <5:
            newarr[i] = 0
        else: 
            newarr[i] = 1
    return newarr
def layer2transform1(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] < 5:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] < 2:
                newarr = torch.cat((newarr,ztensor),dim=0)
            else:
                newarr = torch.cat((newarr,otensor),dim=0)
    return newdata, newarr
def layer2transform2(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] >= 5:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] <7:
                newarr = torch.cat((newarr,ztensor),dim=0)
            else:
                newarr = torch.cat((newarr,otensor),dim=0)
    return newdata, newarr
def layer3transform1(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] <2:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] ==0:
                newarr = torch.cat((newarr,ztensor),dim=0)
            else:
                newarr = torch.cat((newarr,otensor),dim=0)
    return newdata, newarr
def layer3transform2(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        if array[i] >=2 and array[i] <5:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] == 2:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 3:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 4: 
                newarr = torch.cat((newarr,ttensor),dim=0)
    return newdata, newarr
def layer3transform3(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] >=5 and array[i] <7:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] ==5:
                newarr = torch.cat((newarr,ztensor),dim=0)
            else:
                newarr = torch.cat((newarr,otensor),dim=0)
    return newdata, newarr
def layer3transform4(data, array):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    for i in range(100):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        if array[i] >= 7:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            if array[i] == 7:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 8:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 9: 
                newarr = torch.cat((newarr,ttensor),dim=0)
    return newdata, newarr
def test():
    for data, target in modelloader:
        data, target = data.to(device), target.to(device)
        layer2transform2(data,target)
    

In [ ]:
model1 = resnet20(3)
model2 = resnet20(3)
model3 = resnet20(3)
model4 = resnet20(4)
model1.to(device)
model2.to(device)
model3.to(device)
model4.to(device)
criterion1 = nn.CrossEntropyLoss() 
criterion2 = nn.CrossEntropyLoss() 
criterion3 = nn.CrossEntropyLoss() 
criterion4 = nn.CrossEntropyLoss() 
optimizer1 = optim.SGD(model1.parameters(), lr=0.01, weight_decay = 0.001, momentum = 0.9)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01, weight_decay = 0.001, momentum = 0.9)
optimizer3 = optim.SGD(model3.parameters(), lr=0.01, weight_decay = 0.001, momentum = 0.9)
optimizer4 = optim.SGD(model4.parameters(), lr=0.01, weight_decay = 0.001, momentum = 0.9)

In [6]:
model1 = Net()
model2 = Net()
model3 = Net()
model4 = Net()
model5 = Net2()
model6 = Net()
model7 = Net2()
model1.load_state_dict(torch.load("model1"))
model2.load_state_dict(torch.load("model2"))
model3.load_state_dict(torch.load("model3"))
model4.load_state_dict(torch.load("model4"))
model5.load_state_dict(torch.load("model5"))
model6.load_state_dict(torch.load("model6"))
model7.load_state_dict(torch.load("model7"))
model1.to(device)
model2.to(device)
model3.to(device)
model4.to(device)
model5.to(device)
model6.to(device)
model7.to(device)
criterion1 = nn.CrossEntropyLoss() 
criterion2 = nn.CrossEntropyLoss() 
criterion3 = nn.CrossEntropyLoss() 
criterion4 = nn.CrossEntropyLoss() 
criterion5 = nn.CrossEntropyLoss() 
criterion6= nn.CrossEntropyLoss() 
criterion7= nn.CrossEntropyLoss() 
optimizer1 = optim.Adam(model1.parameters(), lr = 0.001)
optimizer2 = optim.Adam(model2.parameters(), lr = 0.001)
optimizer3 = optim.Adam(model3.parameters(), lr = 0.001)
optimizer4 = optim.Adam(model4.parameters(), lr = 0.001)
optimizer5 = optim.Adam(model5.parameters(), lr = 0.001)
optimizer6 = optim.Adam(model6.parameters(), lr = 0.001)
optimizer7 = optim.Adam(model7.parameters(), lr = 0.001)


In [9]:
def trainmodels(epoch): 
    train_loss1 = 0.0
    train_loss2 = 0.0
    train_loss3 = 0.0
    train_loss4 = 0.0
    for data, target in modelloader:
        data, target = data.to(device), target.to(device) 
        initialtarget = layer1transform(target,100)
        data1,target1 =layer2transform1(data,target)
        target1 = target1.type(torch.LongTensor) 
        target1 = target1.to(device)
        data2,target2 =layer2transform2(data,target)
        target2 = target2.type(torch.LongTensor) 
        target2 = target2.to(device)
        data3,target3 =layer2transform3(data,target)
        target3 = target3.type(torch.LongTensor) 
        target3 = target3.to(device)
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        optimizer3.zero_grad()
        optimizer4.zero_grad()
        output1 = model1(data)
        output2 = model2(data1)
        output3 = model3(data2)
        output4 = model4(data3)
        loss1 = criterion1(output1, initialtarget)
        loss1.backward()
        optimizer1.step()
        train_loss1 += loss1.item()*data.size(0)
        loss2 = criterion2(output2,target1)
        loss2.backward()
        optimizer2.step()
        train_loss2 += loss2.item()*data.size(0)
        loss3 = criterion3(output3,target2)
        loss3.backward()
        optimizer3.step()
        train_loss3 += loss3.item()*data.size(0)
        loss4 = criterion4(output4,target3)
        loss4.backward()
        optimizer4.step()
        train_loss4 += loss4.item()*data.size(0)
    train_loss1 = train_loss1/len(modelloader.dataset)
    train_loss2 = train_loss2/len(modelloader.dataset)
    train_loss3 = train_loss3/len(modelloader.dataset)
    train_loss4 = train_loss4/len(modelloader.dataset)
    print('Epoch: {} \tTraining Loss1: {:.6f} \tTraining Loss2: {:.6f} \tTraining Loss3: {:.6f} \tTraining Loss4: {:.6f}'.format(epoch, train_loss1, train_loss2, train_loss3, train_loss4))

In [7]:
def trainmodels2(epoch): 
    train_loss1 = 0.0
    train_loss2 = 0.0
    train_loss3 = 0.0
    train_loss4 = 0.0
    train_loss5 = 0.0
    train_loss6 = 0.0
    train_loss7 = 0.0
    for data, target in modelloader:
        data, target = data.to(device), target.to(device) 
        initialtarget = layer1transform(target,100)
        data1,target1 =layer2transform1(data,target)
        target1 = target1.type(torch.LongTensor) 
        target1 = target1.to(device)
        data2,target2 =layer2transform2(data,target)
        target2 = target2.type(torch.LongTensor) 
        target2 = target2.to(device)
        data3,target3 =layer3transform1(data,target)
        target3 = target3.type(torch.LongTensor) 
        target3 = target3.to(device)
        data4,target4 =layer3transform2(data,target)
        target4 = target4.type(torch.LongTensor) 
        target4 = target4.to(device)
        data5,target5 =layer3transform3(data,target)
        target5 = target5.type(torch.LongTensor) 
        target5 = target5.to(device)
        data6,target6 =layer3transform4(data,target)
        target6 = target6.type(torch.LongTensor) 
        target6 = target6.to(device)
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        optimizer3.zero_grad()
        optimizer4.zero_grad()
        optimizer5.zero_grad()
        optimizer6.zero_grad()
        optimizer7.zero_grad()
        output1 = model1(data)
        output2 = model2(data1)
        output3 = model3(data2)
        output4 = model4(data3)
        output5 = model5(data4)
        output6 = model6(data5)
        output7 = model7(data6)
        loss1 = criterion1(output1, initialtarget)
        loss1.backward()
        optimizer1.step()
        train_loss1 += loss1.item()*data.size(0)
        loss2 = criterion2(output2,target1)
        loss2.backward()
        optimizer2.step()
        train_loss2 += loss2.item()*data.size(0)
        loss3 = criterion3(output3,target2)
        loss3.backward()
        optimizer3.step()
        train_loss3 += loss3.item()*data.size(0)
        loss4 = criterion4(output4,target3)
        loss4.backward()
        optimizer4.step()
        train_loss4 += loss4.item()*data.size(0)
        loss5 = criterion5(output5,target4)
        loss5.backward()
        optimizer5.step()
        train_loss5 += loss5.item()*data.size(0)
        loss6 = criterion6(output6,target5)
        loss6.backward()
        optimizer6.step()
        train_loss6 += loss6.item()*data.size(0)
        loss7 = criterion7(output7,target6)
        loss7.backward()
        optimizer7.step()
        train_loss7 += loss7.item()*data.size(0)
    train_loss1 = train_loss1/len(modelloader.dataset)
    train_loss2 = train_loss2/len(modelloader.dataset)
    train_loss3 = train_loss3/len(modelloader.dataset)
    train_loss4 = train_loss4/len(modelloader.dataset)
    train_loss5 = train_loss5/len(modelloader.dataset)
    train_loss6 = train_loss6/len(modelloader.dataset)
    train_loss7 = train_loss7/len(modelloader.dataset)
    print('Epoch: {} \tTraining Loss1: {:.6f} \tTraining Loss2: {:.6f} \tTraining Loss3: {:.6f} \tTraining Loss4: {:.6f}'.format(epoch, train_loss1, train_loss2, train_loss3, train_loss4))
    print('Epoch: {} \tTraining Loss5: {:.6f} \tTraining Loss6: {:.6f} \tTraining Loss7: {:.6f}'.format(epoch, train_loss5, train_loss6, train_loss7))

In [13]:

n_epochs = [*range(250)] # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss
for epoch in n_epochs:
    model1.train()
    model2.train()
    model3.train()
    model4.train()
   # trainmodels(epoch)
   
    trainmodels2(epoch)
    #test()
        
           

Epoch: 0 	Training Loss1: 0.133103 	Training Loss2: 0.012182 	Training Loss3: 0.018250 	Training Loss4: 0.008977
Epoch: 0 	Training Loss5: 0.049511 	Training Loss6: 0.008273 	Training Loss7: 0.020197
Epoch: 1 	Training Loss1: 0.132874 	Training Loss2: 0.018645 	Training Loss3: 0.015750 	Training Loss4: 0.012056
Epoch: 1 	Training Loss5: 0.070219 	Training Loss6: 0.009168 	Training Loss7: 0.013293
Epoch: 2 	Training Loss1: 0.134812 	Training Loss2: 0.013670 	Training Loss3: 0.018952 	Training Loss4: 0.035210
Epoch: 2 	Training Loss5: 0.061772 	Training Loss6: 0.012157 	Training Loss7: 0.017916
Epoch: 3 	Training Loss1: 0.132104 	Training Loss2: 0.013591 	Training Loss3: 0.017623 	Training Loss4: 0.023480
Epoch: 3 	Training Loss5: 0.055341 	Training Loss6: 0.016188 	Training Loss7: 0.015348
Epoch: 4 	Training Loss1: 0.130764 	Training Loss2: 0.015566 	Training Loss3: 0.015309 	Training Loss4: 0.014456
Epoch: 4 	Training Loss5: 0.059824 	Training Loss6: 0.018144 	Training Loss7: 0.016932


KeyboardInterrupt: 

In [14]:
torch.save(model1.state_dict(), "model1")
torch.save(model2.state_dict(), "model2")
torch.save(model3.state_dict(), "model3")
torch.save(model4.state_dict(), "model4")
torch.save(model5.state_dict(), "model5")
torch.save(model6.state_dict(), "model6")
torch.save(model7.state_dict(), "model7")

In [11]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
train_on_gpu = torch.cuda.is_available()
model1.eval()
model2.eval()
model3.eval()
model4.eval()
for data, target in testloader:
        data, target = data.to(device), target.to(device)
        initialtarget = layer1transform(target,1)
        output1 = model1(data)
        _, pred = torch.max(output1, 1) 
        if pred == 0:
            output2 = model2(data)
            _, pred2 = torch.max(output2, 1)
            if pred2 == 0:
                output3 = model4(data)
                _, predfinal = torch.max(output3, 1) 
            if pred2 == 1:
                output3 = model5(data)
                _, predfinal = torch.max(output3, 1)
                predfinal = predfinal+2
        if pred == 1:
            output2 = model3(data)
            _, pred2 = torch.max(output2, 1)
            if pred2 == 0:
                output3 = model6(data)
                _, predfinal = torch.max(output3, 1) 
                predfinal = predfinal+5
            if pred2 == 1:
                output3 = model7(data)
                _, predfinal = torch.max(output3, 1) 
                predfinal = predfinal+7
        correct_tensor = predfinal.eq(target.data.view_as(predfinal))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        label = target.data[0]
        class_correct[label] += correct.item()
        class_total[label] += 1
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of airplane: 84% (849/1000)
Test Accuracy of automobile: 88% (883/1000)
Test Accuracy of  bird: 73% (738/1000)
Test Accuracy of   cat: 64% (642/1000)
Test Accuracy of  deer: 82% (820/1000)
Test Accuracy of   dog: 73% (733/1000)
Test Accuracy of  frog: 84% (847/1000)
Test Accuracy of horse: 80% (803/1000)
Test Accuracy of  ship: 86% (864/1000)
Test Accuracy of truck: 86% (864/1000)

Test Accuracy (Overall): 80% (8043/10000)


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
train_on_gpu = torch.cuda.is_available()
model1.eval()
model2.eval()
model3.eval()
model4.eval()
for data, target in testloader:
        data, target = data.to(device), target.to(device)
        initialtarget = layer1transform(target,1)
        output1 = model1(data)
        _, pred = torch.max(output1, 1) 
        if pred == 0:
            output2 = model2(data)
            _, pred2 = torch.max(output2, 1)
            predfinal = pred2
        if pred == 1:
            output2 = model3(data)
            _, pred2 = torch.max(output2, 1)
            predfinal = pred2 +3
        correct_tensor = predfinal.eq(target.data.view_as(predfinal))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        label = target.data[0]
        class_correct[label] += correct.item()
        class_total[label] += 1
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))